## Why Governance Matters

Modern AI regulations require:

| Requirement | Regulation | Penalty | How we address it |
|-------------|------------|---------|-------------------|
| Auditability | GDPR Art. 30 | €10M | Full audit trail with metadata |
| Explainability | EU AI Act | €35M | Structured reasoning + sources |
| Risk Management | NIST AI RMF | Contracts cancelled | Policy engines + alerts |
| Bias Reduction | EEOC | Lawsuits | Bias detector + fairness metrics |

```
Data In → Security → Observability → Governance → Trusted Output
          (Notebook 1-3)             (Notebook 4)
```

## Setup

We will reuse the logger and metrics from previous notebooks and add lightweight governance helpers.

In [1]:
import sys
import json
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Optional

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

from observability.logger import ContractAgentLogger
from observability.metrics import MetricsCollector

# Import governance modules
from governance.audit_trail import AuditTrail, AuditRecord
from governance.policy_engine import PolicyEngine, create_default_policies
from governance.explainability import ExplainabilityBuilder
from governance.guardrails import Guardrails, create_default_guardrails

logger = ContractAgentLogger("governance_notebook")
metrics = MetricsCollector()

print("✅ Governance notebook initialized")
print("✅ Imported governance modules:")
print("   • AuditTrail - Immutable audit logging")
print("   • PolicyEngine - Compliance evaluation")
print("   • ExplainabilityBuilder - AI decision explanations")
print("   • Guardrails - Responsible AI checks")

✅ Governance notebook initialized
✅ Imported governance modules:
   • AuditTrail - Immutable audit logging
   • PolicyEngine - Compliance evaluation
   • ExplainabilityBuilder - AI decision explanations
   • Guardrails - Responsible AI checks


## Part 1: Audit Trail Engine

Audit trails must be:
- **Immutable** (append-only)
- **Searchable** (JSON + indices)
- **Tamper-evident** (hash chains)
- **Context-rich** (who, what, when, why)

In [2]:
# Initialize audit trail from governance module
audit_trail = AuditTrail()

print("✅ Audit trail engine ready")
print(f"   Initial hash: {audit_trail.last_hash[:16]}...")
print(f"   Records: {len(audit_trail.records)}")
print("\nFeatures:")
print("  • Append-only (immutable)")
print("  • SHA-256 hash chain")
print("  • Tamper-evident")
print("  • Searchable by request/user/action")

✅ Audit trail engine ready
   Initial hash: 0000000000000000...
   Records: 0

Features:
  • Append-only (immutable)
  • SHA-256 hash chain
  • Tamper-evident
  • Searchable by request/user/action


In [3]:
# Simulate audit entries
request_id = "req-789"
user_id = "legal-analyst-42"

audit_trail.add_record(
    action="contract.upload",
    request_id=request_id,
    user_id=user_id,
    details={
        "filename": "saas_agreement.pdf",
        "size_kb": 420
    },
    status="success"
)

audit_trail.add_record(
    action="security.gateway",
    request_id=request_id,
    user_id=user_id,
    details={
        "pii_detected": False,
        "validators": ["length", "injection", "rate_limit"]
    },
    status="success"
)

audit_trail.add_record(
    action="analysis.generate",
    request_id=request_id,
    user_id=user_id,
    details={
        "contract_type": "SaaS",
        "risks_found": 4,
        "red_flags": 1
    },
    status="success"
)

print("📜 Current Audit Trail:")
print(audit_trail.export_json())

print("\n🔍 Verify Hash Chain:")
is_valid, error = audit_trail.verify_chain()
print(f"   Chain Valid: {is_valid}")
if error:
    print(f"   Error: {error}")

print("\n📊 Statistics:")
stats = audit_trail.get_statistics()
print(f"   Total Records: {stats['total_records']}")
print(f"   Actions: {stats['actions']}")
print(f"   Users: {stats['users']}")

📜 Current Audit Trail:
[
  {
    "timestamp": "2025-11-29T06:44:00.248658",
    "request_id": "req-789",
    "user_id": "legal-analyst-42",
    "action": "contract.upload",
    "details": {
      "filename": "saas_agreement.pdf",
      "size_kb": 420
    },
    "status": "success",
    "previous_hash": "0000000000000000000000000000000000000000000000000000000000000000",
    "record_hash": "1e5ded13226cf9296a6feb367c89bf64a64efcf8d358b2875f0451b3aa0945d4"
  },
  {
    "timestamp": "2025-11-29T06:44:00.248658",
    "request_id": "req-789",
    "user_id": "legal-analyst-42",
    "action": "security.gateway",
    "details": {
      "pii_detected": false,
      "validators": [
        "length",
        "injection",
        "rate_limit"
      ]
    },
    "status": "success",
    "previous_hash": "1e5ded13226cf9296a6feb367c89bf64a64efcf8d358b2875f0451b3aa0945d4",
    "record_hash": "e13c4effc6fcdd028df8e4ebc4ab54e02220e4a36aee3b56c11b3d432c1ae4c1"
  },
  {
    "timestamp": "2025-11-29T06:44:0

## Part 2: Compliance Policy Engine

Policies = executable rules + evidence.

### Sample Policy Catalog
- **GDPR-001**: PII must be redacted before leaving EU region
- **SOC2-LOG**: All decisions must have trace + log + audit record
- **RISK-007**: High risk contracts require manual approval
- **DATA-RET**: Delete analysis after 30 days unless retained

We'll build a lightweight policy engine to evaluate these conditions.

In [4]:
# Initialize policy engine with default policies
policy_engine = create_default_policies()

print("✅ Policy engine ready")
print(f"   Total Policies: {len(policy_engine.policies)}")
print("\n📋 Configured Policies:")

for policy in policy_engine.policies:
    print(f"   • [{policy['severity'].upper()}] {policy['id']}: {policy['description']}")

✅ Policy engine ready
   Total Policies: 5

📋 Configured Policies:
   • [CRITICAL] GDPR-001: PII must be redacted before export
   • [HIGH] SOC2-LOG: All analyses must have trace + audit record
   • [MEDIUM] RISK-007: High-risk contracts require manual approval
   • [LOW] DATA-RET: Analysis retention policy must be set
   • [CRITICAL] AUTH-001: User must be authenticated


In [5]:
# Sample compliance context
context = {
    "request_id": request_id,
    "pii_detected": True,
    "pii_redacted": True,
    "trace_id": "trace-12345",
    "audit_hash": audit_trail.last_hash,
    "logs_written": True,
    "risk_level": "high",
    "human_approved": False
}

print("🧮 Evaluating compliance policies...\n")
result = policy_engine.evaluate(context)

for policy_result in result['results']:
    status = "✅" if policy_result['pass'] else "❌"
    print(f"{status} {policy_result['policy_id']}: {policy_result['description']}")
    print(f"   Evidence: {policy_result['evidence']} (Severity: {policy_result['severity']})")

print(f"\nOverall Compliance Pass: {result['passed']}")

🧮 Evaluating compliance policies...

✅ GDPR-001: PII must be redacted before export
   Evidence: PII detected: True | Redacted: True (Severity: critical)
✅ SOC2-LOG: All analyses must have trace + audit record
   Evidence: Trace: trace-12345 | Audit: bea82418a7c15023f1be6a9db236ad7ebca3c2547cca7b93d1b30754f21adc90 | Logs: True (Severity: high)
❌ RISK-007: High-risk contracts require manual approval
   Evidence: Risk: high | Human approved: False (Severity: medium)
❌ DATA-RET: Analysis retention policy must be set
   Evidence: Retention: None days (Severity: low)
❌ AUTH-001: User must be authenticated
   Evidence: User ID: None (Severity: critical)

Overall Compliance Pass: False


## Part 3: Explainability Engine

Every AI decision must answer two questions:
1. **Why** was this answer produced?
2. **What** evidence supports it?

We'll build a structured explainability object with:
- Reasoning steps
- Confidence breakdown
- Source highlights
- Human-readable summary

In [6]:
# Build explainability using the governance module
sample_explainability = ExplainabilityBuilder.build_explanation(
    contract_type="SaaS Agreement",
    classification_reasoning="Contains subscription language, uptime SLAs, and multi-tenant references.",
    risks=[
        {"risk": "No explicit data portability clause", "severity": "HIGH"},
        {"risk": "Auto-renewal without notification", "severity": "MEDIUM"}
    ],
    confidence=0.87,
    key_clauses=[
        "Section 3.2 Availability SLA",
        "Section 5.1 Term & Renewal"
    ]
)

print("📊 Structured Explanation:")
print(json.dumps(sample_explainability, indent=2))

print("\n" + "=" * 80)
print("HUMAN-READABLE FORMAT:")
print(ExplainabilityBuilder.format_for_human(sample_explainability))

📊 Structured Explanation:
{
  "summary": "Classified as SaaS Agreement with confidence 87%.",
  "reasoning_steps": [
    "Analyzed clause structure and keywords.",
    "Applied contract taxonomy to classify as SaaS Agreement.",
    "Evaluated obligations and risk statements.",
    "Compared against compliance policy requirements."
  ],
  "llm_reasoning": "Contains subscription language, uptime SLAs, and multi-tenant references.",
  "confidence_breakdown": {
    "contract_type": 0.87,
    "risk_assessment": 0.783,
    "clause_extraction": 0.8265
  },
  "evidence": {
    "key_risks": [
      {
        "risk": "No explicit data portability clause",
        "severity": "HIGH"
      },
      {
        "risk": "Auto-renewal without notification",
        "severity": "MEDIUM"
      }
    ],
    "clauses": [
      "Section 3.2 Availability SLA",
      "Section 5.1 Term & Renewal"
    ],
    "risk_count": 2,
    "high_severity_risks": 1
  },
  "human_review_required": true,
  "created_at": "202

## Part 4: Responsible AI Guardrails

Guardrails enforce ethical, legal, and brand standards before outputs reach humans.

### Guardrail Categories
1. **Content safety** (harmful/biased language)
2. **Policy compliance** (industry rules)
3. **Fairness checks** (protected classes)
4. **Result confidence** (fallback to human)

In [7]:
# Initialize guardrails from governance module
guardrails = create_default_guardrails()

print("✅ Guardrails initialized")
print(f"   Confidence threshold: {guardrails.confidence_threshold:.0%}")
print("\n🛡️ Enabled Checks:")
print(f"   • Content Safety: {guardrails.enable_content_safety}")
print(f"   • Bias Detection: {guardrails.enable_bias_detection}")
print(f"   • Confidence Check: {guardrails.enable_confidence_check}")

# Test with sample analysis
test_report = {
    "confidence": 0.58,
    "contract_type": "SaaS Agreement",
    "risks": [
        {"risk": "No data portability clause", "severity": "HIGH"}
    ],
    "summary": "Analysis complete"
}

print("\n🧪 Running guardrails...")
guardrail_result = guardrails.run_guardrails(test_report)

print(f"\nPassed: {guardrail_result['passed']}")
print(f"Issues Found: {len(guardrail_result['issues'])}")
print(f"Requires Human Review: {guardrail_result['requires_human_review']}")

if guardrail_result['issues']:
    print("\n⚠️  Issues Detected:")
    for issue in guardrail_result['issues']:
        print(f"   • [{issue['severity'].upper()}] {issue['type']}: {issue['message']}")

if guardrail_result['actions']:
    print("\n📋 Recommended Actions:")
    for action in guardrail_result['actions']:
        print(f"   → {action}")

Low confidence: 58.0%


✅ Guardrails initialized
   Confidence threshold: 60%

🛡️ Enabled Checks:
   • Content Safety: True
   • Bias Detection: True
   • Confidence Check: True

🧪 Running guardrails...

Passed: False
Issues Found: 1
Requires Human Review: True

⚠️  Issues Detected:
   • [HIGH] low_confidence: Confidence 58.0% below threshold 60.0%

📋 Recommended Actions:
   → Escalate to human reviewer


## Part 5: Governance-Oriented LangGraph Flow

We'll simulate the final stage of the contract agent that:
1. Consumes analysis results
2. Generates explainability + audit
3. Runs compliance policies
4. Applies responsible AI guardrails
5. Dispatches to user or escalates to human

In [8]:
def governance_orchestrator(analysis: Dict[str, Any]) -> Dict[str, Any]:
    """
    Complete governance pipeline for contract analysis.
    
    Steps:
    1. Generate explainability
    2. Create audit record
    3. Evaluate compliance policies
    4. Run guardrails
    5. Make final decision
    """
    request_id = analysis['request_id']
    user_id = analysis['user_id']

    logger.set_request_context(request_id=request_id, user_id=user_id)

    # 1. Explainability
    explainability = ExplainabilityBuilder.build_explanation(
        contract_type=analysis['contract_type'],
        classification_reasoning=analysis['classification_reasoning'],
        risks=analysis['risks'],
        confidence=analysis['confidence'],
        key_clauses=analysis['key_clauses']
    )

    # 2. Audit
    audit_record = audit_trail.add_record(
        action="governance.finalize",
        request_id=request_id,
        user_id=user_id,
        details={
            "contract_type": analysis['contract_type'],
            "confidence": analysis['confidence'],
            "risks": len(analysis['risks'])
        },
        status="success"
    )

    # 3. Compliance policies
    policy_context = {
        "pii_detected": analysis.get('pii_detected', False),
        "pii_redacted": analysis.get('pii_redacted', False),
        "trace_id": analysis.get('trace_id'),
        "audit_hash": audit_record.record_hash,
        "logs_written": True,
        "risk_level": analysis.get('risk_level', 'low'),
        "human_approved": analysis.get('human_approved', False),
        "user_id": user_id,
        "retention_days": 30
    }
    policy_outcome = policy_engine.evaluate(policy_context)

    # 4. Guardrails
    guardrail_outcome = guardrails.run_guardrails({
        "confidence": analysis['confidence'],
        "contract_type": analysis['contract_type'],
        "risks": analysis['risks'],
        "summary": explainability['summary']
    })

    # 5. Final decision
    final_decision = "approved"
    if not policy_outcome['passed'] or not guardrail_outcome['passed']:
        final_decision = "escalate_to_human"
        metrics.record_security_violation(violation_type="governance_block")

    logger.info("Governance decision", extra={
        "decision": final_decision,
        "policy_pass": policy_outcome['passed'],
        "guardrail_pass": guardrail_outcome['passed']
    })

    return {
        "decision": final_decision,
        "explainability": explainability,
        "policy_outcome": policy_outcome,
        "guardrail_outcome": guardrail_outcome,
        "audit_hash": audit_record.record_hash,
        "request_id": request_id
    }

# Test with sample analysis
sample_analysis = {
    "request_id": "req-999",
    "user_id": "legal-ops",
    "contract_type": "SaaS Agreement",
    "classification_reasoning": "Recurring subscription fees, uptime clauses, support tiers.",
    "risks": [
        {"risk": "No SLA penalties", "severity": "HIGH"}
    ],
    "confidence": 0.75,
    "key_clauses": ["Section 4 Payment Terms", "Section 5 Service Levels"],
    "pii_detected": False,
    "pii_redacted": False,
    "trace_id": "trace-abc123",
    "risk_level": "high",
    "human_approved": False
}

print("🔄 Running Governance Orchestrator...\n")
final_output = governance_orchestrator(sample_analysis)

print("=" * 80)
print(f"FINAL DECISION: {final_output['decision'].upper()}")
print("=" * 80)
print(f"\n✅ Policy Compliance: {final_output['policy_outcome']['passed']}")
print(f"   Policies Passed: {final_output['policy_outcome']['policies_passed']}/{final_output['policy_outcome']['total_policies']}")

print(f"\n✅ Guardrails: {final_output['guardrail_outcome']['passed']}")
print(f"   Issues Found: {len(final_output['guardrail_outcome']['issues'])}")

print(f"\n📝 Audit Hash: {final_output['audit_hash'][:32]}...")

logger.clear_request_context()

🔄 Running Governance Orchestrator...

2025-11-29 12:16:48,111 - governance_notebook - INFO - Governance decision
FINAL DECISION: ESCALATE_TO_HUMAN

✅ Policy Compliance: False
   Policies Passed: 4/5

✅ Guardrails: True
   Issues Found: 0

📝 Audit Hash: 2b1f7c64be1c661d0b15ca616ddffbbb...


## Governance Dashboard Snapshot

Combine audit logs, policy outcomes, and guardrail events for leadership visibility.

In [9]:
from collections import Counter

# Simulate governance events
policy_decisions = ["approved", "escalate", "approved", "approved", "escalate"]
guardrail_flags = ["none", "bias", "confidence", "none", "content"]
human_reviews = sum(1 for d in policy_decisions if d == "escalate")

print("📊 GOVERNANCE OVERVIEW")
print("=" * 80)
print(f"Total Analyses:        {len(policy_decisions)}")
print(f"Auto-Approved:        {policy_decisions.count('approved')}")
print(f"Escalated to Humans: {human_reviews}")
print(f"Guardrail Flags:      {Counter(guardrail_flags)}")
print("=" * 80)
print("Use this data for quarterly governance reviews and board reporting.")

📊 GOVERNANCE OVERVIEW
Total Analyses:        5
Auto-Approved:        3
Escalated to Humans: 2
Guardrail Flags:      Counter({'none': 2, 'bias': 1, 'confidence': 1, 'content': 1})
Use this data for quarterly governance reviews and board reporting.


## Final Checklist

| Capability | Status | Evidence |
|------------|--------|----------|
| Audit Trail | ✅ | Hash-chained JSON log |
| Compliance Policies | ✅ | GDPR/SOC2/Risk rules |
| Explainability | ✅ | Structured reasoning object |
| Responsible AI | ✅ | Guardrail issues + decisions |
| Escalation Path | ✅ | Auto human handoff |
| Metrics | ✅ | Compliance + security counters |

**Regulatory Alignment**
- EU AI Act (Risk management, transparency, human oversight)
- GDPR (Records of processing, privacy by design)
- SOC 2 (Security + Privacy trust principles)
- ISO 42001 (AI management system controls)
- NIST AI RMF (Govern + Map + Measure + Manage)

---

| Hour | Theme | Notebook | Outcome |
|------|-------|----------|---------|
| 1 | LangGraph Foundations | 01_introduction_and_setup | Working contract agent |
| 2 | Observability | 02_observability_and_monitoring | Traces, metrics, logs |
| 3 | Security & PII | 03_security_and_pii_detection | Safe processing pipeline |
| 4 | Governance & Responsible AI | 04_governance_and_responsible_ai | Trustworthy delivery |

